<h2>Quick Start: Creating Sample-wise Unlearnable Examples</h2>

<h3>Prepare Data</h3>

In [9]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from Get_data import get_data
from Make_label import make_label
from torch.utils.data import DataLoader, TensorDataset
# # Prepare Dataset---------------------------------------------------------------------------------------
# train_transform = [
#     transforms.ToTensor()
# ]
# test_transform = [
#     transforms.ToTensor()
# ]
# train_transform = transforms.Compose(train_transform)
# test_transform = transforms.Compose(test_transform)

# clean_train_dataset = datasets.CIFAR10(root='../datasets', train=True, download=True, transform=train_transform)
# clean_test_dataset = datasets.CIFAR10(root='../datasets', train=False, download=True, transform=test_transform)
#--------------------------------------------------------------------------------------------------------
# clean_train_loader = DataLoader(dataset=clean_train_dataset, batch_size=512,
#                                 shuffle=False, pin_memory=True,
#                                 drop_last=False, num_workers=12)
# clean_test_loader = DataLoader(dataset=clean_test_dataset, batch_size=512,
#                                 shuffle=False, pin_memory=True,
#                                 drop_last=False, num_workers=12)
#--------------------------------------------------------------------------------------------------------

#   插入我们自己的dataset
# 获取数据
start_num = 0
end_num = 70000 #   start_num 到 7W作为训练集 ，剩余是验证集
target = 9                                              # 先试第9个字节
file_addr = 'E:\\HZY\\matlab\\preprocess\\R_10W_halfbyte32.mat'  # 数据位置
# file_addr = 'C:/Users/Administrator/Desktop/并行测试/R_99000_train.mat'  # 数据位置
X, plaintext, key, cipher = get_data(file_addr)
label = make_label(plaintext, key, target)
X = torch.FloatTensor(X)  # 数据类型转换
X=X.view(-1,1,2940,1)   #   重塑X为[batchsize,1,2940,1]的形状
label = torch.LongTensor(label)  # 数据类型转换
db_train = TensorDataset(X[start_num:end_num], label[start_num:end_num])  # 封装数据集
#train_DataLoader = DataLoader(db_train, batch_size=batch_size, shuffle=True, num_workers=0)
db_val = TensorDataset(X[end_num:99000], label[end_num:99000])
#val_DataLoader = DataLoader(db_val, batch_size=batch_size, shuffle=False, num_workers=0)
#dataloader = {'train': train_DataLoader, 'val':val_DataLoader}

clean_train_loader = DataLoader(dataset=db_train, batch_size=64,
                                shuffle=False, pin_memory=True,
                                drop_last=False, num_workers=12)
clean_test_loader = DataLoader(dataset=db_val, batch_size=64,
                                shuffle=False, pin_memory=True,
                                drop_last=False, num_workers=12)


<h3>Prepare Model</h3>

In [2]:
from models.ResNet import ResNet18
import toolbox

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

base_model = ResNet18()
base_model = base_model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=base_model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)

noise_generator = toolbox.PerturbationTool(epsilon=0.03137254901960784, num_steps=20, step_size=0.0031372549019607846)

c:\ProgramData\Anaconda3\envs\UE\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h3>Generate Error-Minimizing Noise</h3>

In [3]:
from tqdm import tqdm

noise = torch.zeros([50000, 3, 32, 32])
data_iter = iter(clean_train_loader)
condition = True
train_idx = 0

while condition:
    # optimize theta for M steps
    base_model.train()
    for param in base_model.parameters():
        param.requires_grad = True
    for j in range(0, 10):
        try:
            (images, labels) = next(data_iter)
        except:
            train_idx = 0
            data_iter = iter(clean_train_loader)
            (images, labels) = next(data_iter)
        
        for i, _ in enumerate(images):
            # Update noise to images
            images[i] += noise[train_idx]
            train_idx += 1
        images, labels = images.cuda(), labels.cuda()
        base_model.zero_grad()
        optimizer.zero_grad()
        logits = base_model(images)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(base_model.parameters(), 5.0)
        optimizer.step()
    
    # Perturbation over entire dataset
    idx = 0
    for param in base_model.parameters():
        param.requires_grad = False
    for i, (images, labels) in tqdm(enumerate(clean_train_loader), total=len(clean_train_loader)):
        batch_start_idx, batch_noise = idx, []
        for i, _ in enumerate(images):
            # Update noise to images
            batch_noise.append(noise[idx])
            idx += 1
        batch_noise = torch.stack(batch_noise).cuda()
        
        # Update sample-wise perturbation
        base_model.eval()
        images, labels = images.cuda(), labels.cuda()
        perturb_img, eta = noise_generator.min_min_attack(images, labels, base_model, optimizer, criterion, 
                                                          random_noise=batch_noise)
        for i, delta in enumerate(eta):
            noise[batch_start_idx+i] = delta.clone().detach().cpu()
        
    # Eval stop condition
    eval_idx, total, correct = 0, 0, 0
    for i, (images, labels) in enumerate(clean_train_loader):
        for i, _ in enumerate(images):
            # Update noise to images
            images[i] += noise[eval_idx]
            eval_idx += 1
        images, labels = images.cuda(), labels.cuda()
        with torch.no_grad():
            logits = base_model(images)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    print('Accuracy %.2f' % (acc*100))
    if acc > 0.99:
        condition=False      


100%|██████████| 98/98 [03:05<00:00,  1.89s/it]


Accuracy 8.13


100%|██████████| 98/98 [03:06<00:00,  1.91s/it]


Accuracy 11.89


100%|██████████| 98/98 [03:07<00:00,  1.91s/it]


...

Accuracy 96.87


100%|██████████| 98/98 [03:07<00:00,  1.92s/it]


Accuracy 97.75


100%|██████████| 98/98 [03:07<00:00,  1.91s/it]


Accuracy 99.72


In [4]:
# Examine the noise
print(noise)

tensor([[[[ 2.5098e-02,  3.1373e-02,  2.8235e-02,  ...,  3.1373e-02,
            3.1373e-02,  3.1373e-02],
          [-9.4118e-03,  1.8824e-02, -1.2549e-02,  ..., -2.8235e-02,
           -2.8235e-02, -2.8235e-02],
          [-2.1961e-02, -3.1373e-02, -3.1373e-02,  ..., -3.1373e-02,
           -3.1373e-02, -3.1373e-02],
          ...,
          [ 3.1370e-03, -2.1961e-02, -2.8235e-02,  ...,  6.2747e-03,
            1.2549e-02,  2.5098e-02],
          [-3.1370e-03,  9.4119e-03,  9.4119e-03,  ...,  2.3842e-07,
           -2.5098e-02, -3.1373e-02],
          [-3.1370e-03,  3.1375e-03, -1.5686e-02,  ..., -6.2742e-03,
           -2.3842e-07, -1.5686e-02]],

         [[-3.1373e-02, -5.9605e-08, -2.8235e-02,  ...,  3.1371e-03,
           -1.5686e-02, -2.1961e-02],
          [-3.1373e-02,  1.2549e-02,  6.2745e-03,  ...,  3.1373e-02,
            3.1373e-02,  3.1373e-02],
          [ 3.1373e-02,  2.5098e-02,  2.5098e-02,  ...,  3.1373e-02,
            3.1373e-02,  3.1373e-02],
          ...,
     

<h3>Creat Unlearnable Dataset</h3>

In [5]:
import numpy as np

# Add standard augmentation
train_transform = [
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
]
train_transform = transforms.Compose(train_transform)
clean_train_dataset = datasets.CIFAR10(root='../datasets', train=True, download=True, transform=train_transform)
unlearnable_train_dataset = datasets.CIFAR10(root='../datasets', train=True, download=True, transform=train_transform)

perturb_noise = noise.mul(255).clamp_(0, 255).permute(0, 2, 3, 1).to('cpu').numpy()
unlearnable_train_dataset.data = unlearnable_train_dataset.data.astype(np.float32)
for i in range(len(unlearnable_train_dataset)):
    unlearnable_train_dataset.data[i] += perturb_noise[i]
    unlearnable_train_dataset.data[i] = np.clip(unlearnable_train_dataset.data[i], a_min=0, a_max=255)
unlearnable_train_dataset.data = unlearnable_train_dataset.data.astype(np.uint8)




Files already downloaded and verified
Files already downloaded and verified


<h3>Visualize Clean Images, Error-Minimizing Noise, Unlearnable Images</h3>

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

def imshow(img):
    fig = plt.figure(figsize=(8, 8), dpi=80, facecolor='w', edgecolor='k')
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def get_pairs_of_imgs(idx):
    clean_img = clean_train_dataset.data[idx]
    unlearnable_img = unlearnable_train_dataset.data[idx]
    clean_img = torchvision.transforms.functional.to_tensor(clean_img)
    unlearnable_img = torchvision.transforms.functional.to_tensor(unlearnable_img)

    x = noise[idx]
    x_min = torch.min(x)
    x_max = torch.max(x)
    noise_norm = (x - x_min) / (x_max - x_min)
    noise_norm = torch.clamp(noise_norm, 0, 1)
    return [clean_img, noise_norm, unlearnable_img]
    
selected_idx = [random.randint(0, 50000) for _ in range(3)]
img_grid = []
for idx in selected_idx:
    img_grid += get_pairs_of_imgs(idx)
    

imshow(torchvision.utils.make_grid(torch.stack(img_grid), nrow=3, pad_value=255))



<h3>Train ResNet18 on Clean Dataset</h3>

In [10]:
from util import AverageMeter
from tqdm import tqdm   #   显示进度条
model = ResNet18()
model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=0)

clean_loader = DataLoader(dataset=clean_train_dataset, batch_size=128,
                                shuffle=True, pin_memory=True,
                                drop_last=False, num_workers=12)


for epoch in range(30):
    # Train
    model.train()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    pbar = tqdm(clean_loader, total=len(clean_loader))  # pbar，可迭代对象，引入包是为了显示进度条。同时也作为输入
    for images, labels in pbar:
        images, labels = images.cuda(), labels.cuda()
        model.zero_grad()
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        
        _, predicted = torch.max(logits.data, 1)
        acc = (predicted == labels).sum().item()/labels.size(0)
        acc_meter.update(acc)
        loss_meter.update(loss.item())
        pbar.set_description("Acc %.2f Loss: %.2f" % (acc_meter.avg*100, loss_meter.avg))
    scheduler.step()
    # Eval
    model.eval()
    correct, total = 0, 0
    for i, (images, labels) in enumerate(clean_test_loader):
        images, labels = images.cuda(), labels.cuda()
        with torch.no_grad():
            logits = model(images)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    tqdm.write('Clean Accuracy %.2f\n' % (acc*100))
            

Acc 39.76 Loss: 1.66: 100%|██████████| 391/391 [01:04<00:00,  6.08it/s]


RuntimeError: Given groups=1, weight of size [64, 3, 3, 3], expected input[64, 1, 2940, 1] to have 3 channels, but got 1 channels instead

<h3>Train ResNet18 on Unlearnable Dataset</h3>

In [7]:
from util import AverageMeter

model = ResNet18()
model = model.cuda()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.1, weight_decay=0.0005, momentum=0.9)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30, eta_min=0)

unlearnable_loader = DataLoader(dataset=unlearnable_train_dataset, batch_size=128,
                                shuffle=True, pin_memory=True,
                                drop_last=False, num_workers=12)


for epoch in range(30):
    # Train
    model.train()
    acc_meter = AverageMeter()
    loss_meter = AverageMeter()
    pbar = tqdm(unlearnable_loader, total=len(unlearnable_loader))
    for images, labels in pbar:
        images, labels = images.cuda(), labels.cuda()
        model.zero_grad()
        optimizer.zero_grad()
        logits = model(images)
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        
        _, predicted = torch.max(logits.data, 1)
        acc = (predicted == labels).sum().item()/labels.size(0)
        acc_meter.update(acc)
        loss_meter.update(loss.item())
        pbar.set_description("Acc %.2f Loss: %.2f" % (acc_meter.avg*100, loss_meter.avg))
    scheduler.step()
    # Eval
    model.eval()
    correct, total = 0, 0
    for i, (images, labels) in enumerate(clean_test_loader):
        images, labels = images.cuda(), labels.cuda()
        with torch.no_grad():
            logits = model(images)
            _, predicted = torch.max(logits.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = correct / total
    tqdm.write('Clean Accuracy %.2f\n' % (acc*100))
            

  0%|          | 0/391 [00:00<?, ?it/s]

Clean Accuracy 35.16



  0%|          | 0/391 [00:00<?, ?it/s]

Clean Accuracy 22.21



  0%|          | 0/391 [00:00<?, ?it/s]

...

Clean Accuracy 23.19



  0%|          | 0/391 [00:00<?, ?it/s]

Clean Accuracy 22.83



Acc 99.94 Loss: 0.00: 100%|██████████| 391/391 [00:20<00:00, 19.46it/s]


Clean Accuracy 23.60

